In [1]:
import cv2
import numpy as np
import tkinter as tk
import imutils
from tkinter import messagebox
import winsound
import datetime
import matplotlib.pyplot as plt

def sound_alarm():
    # Play a beep sound
    frequency = 6000  # Set frequency 
    duration = 1000  # Set duration to 1000 ms (1 second)
    winsound.Beep(frequency, duration)
    pass

def popup_alert():
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    tk.messagebox.showinfo("Object Detected", "An object has been detected!")  # Show the popup message
    pass

# Load pre-trained model for object detection
net = cv2.dnn.readNet("yolov3_training_2000.weights", "yolov3_testing.cfg")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_DEFAULT)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
classes = ["Weapon","knife"]

# Initialize video capture
# cap = cv2.VideoCapture("pistol-2.mp4")
cap = cv2.VideoCapture(0)

while True:
    _, img = cap.read()
    height, width, channels = img.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    outs = net.forward(output_layers)

    # Showing information on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    cv2.putText(img, datetime.datetime.now().strftime("%A %d %B %Y %I:%M:%S %p"),
                # (10, img.shape[0] - 30),
                # cv2.FONT_HERSHEY_SIMPLEX,
                # 0.3, (0, 0, 255), 2)
                (10, img.shape[0] - 20),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.7, (0, 0, 255), 1)
    if indexes == 0: 
        print("Detected")
        current_time = datetime.datetime.now().strftime("%A %d %B %Y %I:%M:%S %p")
        # cv2.putText(img, "Detection Time: " + current_time,
        #             (10, img.shape[0] - 50),
        #             cv2.FONT_HERSHEY_SIMPLEX,
        #             0.7, (0, 0, 255), 2)
        
        print(current_time)
        
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                color = colors[class_ids[i]]
                cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
                cv2.putText(img, label, (x, y + 30), cv2.FONT_HERSHEY_PLAIN, 3, color, 3)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.show()
        sound_alarm()
        popup_alert()  # Show the popup alert message
        break

    cv2.imshow("Image", img)
    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()


NameError: name 'cap' is not defined